# Setup

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import random

import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, GPT2LMHeadModel

import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/marcoguerini/CONAN/master/Multitarget-CONAN/Multitarget-CONAN.csv')

In [ ]:
import pickle
with open('file.pkl', 'rb') as f:
    rats = pickle.load(f)

In [ ]:
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

entities = []
for i in tqdm(range(5003)):
    doc = nlp(df["HATE_SPEECH"][i])
    l = ""
    if len(doc.ents) > 0:
        for ent in doc.ents:
            l += "_".join(ent.text.split(" ")) + "_" + ent.label_ + " "
    entities.append(l)

df = df.assign(textentities = entities)

In [ ]:
def find_top_k(tokens):
  words = -1 * round(0.30 * (len(tokens['Tokens']) - 2))
  ind = sorted(np.argpartition(tokens['Rationale'], words)[words:])
  c = [tokens['Tokens'][index] for index in ind]
  return ' '.join(c).replace(' ##', "")

In [ ]:
from tqdm import tqdm

rationales = []
for i in tqdm(range(5003)):
  rationales.append(find_top_k(rats[i]))

In [ ]:
df.dropna(inplace=True) #remove NA values
df["combined"] = df["HATE_SPEECH"] + "<|eohs|>" + df["textentities"] + "<|eone|>"
#df["combined"] = df["HATE_SPEECH"] + "<|eohs|>" + rationales + "<|eor|>" #+ #df["textentities"] + "<|eone|>"
data = [(df["combined"][i], df["COUNTER_NARRATIVE"][i]) for i in range(5003)]
data2 = df[["HATE_SPEECH", "TARGET"]]
data3 = df["COUNTER_NARRATIVE"]

In [ ]:
data2 = df[["HATE_SPEECH", "TARGET"]]
data3 = df["COUNTER_NARRATIVE"]

In [ ]:
df["TARGET"].value_counts()

In [ ]:
data[100]

In [ ]:
batch_size = 2

In [ ]:
from sklearn.model_selection import train_test_split
t1, v1, t2, v2 = train_test_split(data2, data3, test_size=0.1)

In [ ]:
print(v1["HATE_SPEECH"].tolist()[491])

In [ ]:
!pip install evaluate
import evaluate
bleu = evaluate.load("bleu")

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
device = torch.device("cuda")
model.cuda()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
model = GPT2LMHeadModel.from_pretrained('drive/MyDrive/model-rs-ner')
tokenizer = GPT2Tokenizer.from_pretrained('drive/MyDrive/model-rs-ner')
model.to(device)

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
val["TARGET"].value_counts()

In [ ]:
analyzer = SentimentIntensityAnalyzer()
total = {"MUSLIMS":[], "POC":[], "MIGRANTS":[], "LGBT+":[], "WOMEN":[], "JEWS":[], "other":[], "DISABLED":[]}

for i,out in enumerate(true_out):
    vs = analyzer.polarity_scores(out)
    total[val["TARGET"][i]].append(vs['compound'])

In [ ]:
total = {"MUSLIMS":[], "POC":[], "MIGRANTS":[], "LGBT+":[], "WOMEN":[], "JEWS":[], "other":[], "DISABLED":[]}
val = v1.reset_index()

for i,out in enumerate(true_out):
    total[val["TARGET"][i]].append(out)

    #print(vs)
    #print("{:-<65} {}".format(out, str(vs)))
#print(total / 501)

In [ ]:
tox = {}
for id in total.keys():
  results = toxicity.compute(predictions = total[id])
  tox[id] = sum(results['toxicity']) / len(results['toxicity'])
results = toxicity.compute(predictions=true_out)

In [ ]:
tox

In [ ]:
for key in total.keys():
  print(key + ": " + str(sum(total[key]) / len(total[key])))

In [ ]:
val["TARGET"]

In [ ]:
v1.reset_index()

In [ ]:
from tqdm import tqdm

#model.eval()

outputs = []
val = v1.reset_index()
valseq = val["HATE_SPEECH"]
for seq in tqdm(valseq):
  #v1.reset_index()
  prompt =  "<|startoftext|>" + seq + "<|eohs|>"

  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=1, pad_token_id=tokenizer.eos_token_id
                                  )

  
  for i, sample_output in enumerate(sample_outputs):
    #print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True).split("<|eohs|>")[1])

In [ ]:
true_out = [output.split("<|eone|>")[-1] for output in outputs]

In [ ]:
from nltk.translate import bleu_score
print(bleu_score.corpus_bleu(v2.tolist(),
    true_out,
    weights=(1, 0, 0, 0),
    smoothing_function=None,
    auto_reweigh=False,
))

In [ ]:
true_out = [output.split("<|eone|>")[-1] for output in outputs]

In [ ]:
true_out

In [ ]:
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(0, 0, 1, 0),
    smoothing_function=None,
    auto_reweigh=False,
))

In [ ]:
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(0, 0, 0, 1),
    smoothing_function=None,
    auto_reweigh=False,
))

In [ ]:
bleu.compute(predictions=outputs, references=v2.tolist())

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
#!pip install rouge_score
rouge = evaluate.load('rouge')
r = rouge.compute(predictions=true_out, references=v2.tolist())
#!pip install meteor_score
meteor = evaluate.load('meteor')
m = meteor.compute(predictions=true_out, references=v2.tolist())


In [ ]:
#!pip install evaluate
import evaluate
toxicity = evaluate.load("toxicity", module_type="measurement")

In [ ]:
!pip install transformers

In [ ]:
for i,out in enumerate(true_out):
    print(out)
    print(v1["HATE_SPEECH"].reset_index()[491])

In [ ]:
print(v1["HATE_SPEECH"].reset_index()[490:])

In [ ]:
results = toxicity.compute(predictions=true_out)

In [ ]:
for i in range(len(results['toxicity'])):
  if results['toxicity'][i] > 0.2:
    print(str(results['toxicity'][i]) + " " + true_out[i])

In [ ]:
print(results['max_toxicity'])

In [ ]:
r

In [ ]:
m

In [ ]:
r

In [ ]:
r

In [ ]:
m

In [ ]:
true_out = [output.split("<|eone|>")[-1] for output in outputs]

In [ ]:
print(outputs)

In [ ]:
files.download("test.zip")

In [ ]:
!cp test.zip gdrive/MyDrive/models

In [ ]:
!ls drive/MyDrive/models

In [ ]:
!zip -r test.zip /content/model_save/

In [ ]:
from google.colab import files
files.download("test.zip")